In [1]:
import sys
import os

In [2]:
# 현재 스크립트 파일의 디렉토리를 가져오기
script_dir = os.path.dirname(os.path.abspath("parse.ipynb"))

# 상위 폴더의 경로 계산
parent_dir = os.path.join(script_dir, '..')

# 상위 폴더의 경로를 sys.path에 추가
sys.path.append(parent_dir)

In [3]:
# 이제 sven 폴더에 있는 utils.py를 import 할 수 있음
from sven.utils import parse_diff

/Users/jongko/Library/Mobile Documents/com~apple~CloudDocs/Fall 2023/AI & Deep Learning Framework/Term Project/Code Generation/sven/.venv/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
file_name = "temp.py"
src_before = "import numpy as np \n\n   def sum():\n    arr = np.array([[1, 2, 3], [4, 5, 6]])\n      result = np.sum(arr, axis=0)  # Sum in column direction (axis=0)\n    print(result)"
src_after = "import pandas as pd \n\n   def sum():\n    df = pd.DataFrame([[1, 2, 3], [4, 5, 6]])\n      result = df.sum(axis=0)  # Sum in column direction (axis=0)\n    print(result)"
diff = """
@@ -4,2 +4,2 @@ def sum():

-    arr = np.array([[1, 2, 3], [4, 5, 6]])
+    df = pd.DataFrame([[1, 2, 3], [4, 5, 6]])

"""

In [39]:
# result = parse_diff(file_name, src_before, src_after, diff)

In [40]:
# If check the result, No "vul_type", "commit_link" otherwise, same.
# print(result)

In [43]:
file_name = "temp.py"
numpy_code = """
import numpy as np 

def sum():
    arr = np.array([[1, 2, 3], [4, 5, 6]])
    result = np.sum(arr, axis=0)  # Sum in column direction (axis=0)
    print(result)
"""

pandas_code = """
import pandas as pd 

def sum():
    df = pd.DataFrame([[1, 2, 3], [4, 5, 6]])
    result = df.sum(axis=0)  # Sum in column direction (axis=0)
    print(result)
"""

In [44]:
import difflib

# Using unified_diff

# diff_result = difflib.unified_diff(numpy_code.splitlines(), pandas_code.splitlines(), fromfile='', tofile='', lineterm='', n=0)
# diff_result = difflib.unified_diff(numpy_code.splitlines(), pandas_code.splitlines(), fromfile='', tofile='', lineterm='')
diff_result = difflib.unified_diff(numpy_code.split('\n'), pandas_code.split('\n'), fromfile='', tofile='', lineterm='')

next(diff_result)
next(diff_result)

diff = '\n'.join(diff_result)

print(diff)

@@ -1,8 +1,8 @@
 
-    import pandas as pd 
+import pandas as pd 
 
 def sum():
-    arr = np.array([[1, 2, 3], [4, 5, 6]])
-    result = np.sum(arr, axis=0)  # Sum in column direction (axis=0)
+    df = pd.DataFrame([[1, 2, 3], [4, 5, 6]])
+    result = df.sum(axis=0)  # Sum in column direction (axis=0)
     print(result)
 


# Generate diff

In [64]:
import difflib

def generate_diff(numpy_code, pandas_code):
    diff_result = difflib.unified_diff(numpy_code.split('\n'), pandas_code.split('\n'), fromfile='', tofile='', lineterm='')

    next(diff_result)
    next(diff_result)

    diff = '\n'.join(diff_result)
    return diff

In [13]:
diff = generate_diff(numpy_code, pandas_code)

In [14]:
result = parse_diff(file_name, numpy_code, pandas_code, diff)

In [15]:
print(result)

[{'func_name': 'sum', 'func_src_before': 'def sum():\n    arr = np.array([[1, 2, 3], [4, 5, 6]])\n    result = np.sum(arr, axis=0)  # Sum in column direction (axis=0)\n    print(result)', 'func_src_after': 'def sum():\n    df = pd.DataFrame([[1, 2, 3], [4, 5, 6]])\n    result = df.sum(axis=0)  # Sum in column direction (axis=0)\n    print(result)', 'line_changes': {'deleted': [{'line_no': 2, 'char_start': 11, 'char_end': 54, 'line': '    arr = np.array([[1, 2, 3], [4, 5, 6]])\n'}, {'line_no': 3, 'char_start': 54, 'char_end': 123, 'line': '    result = np.sum(arr, axis=0)  # Sum in column direction (axis=0)\n'}], 'added': [{'line_no': 2, 'char_start': 11, 'char_end': 57, 'line': '    df = pd.DataFrame([[1, 2, 3], [4, 5, 6]])\n'}, {'line_no': 3, 'char_start': 57, 'char_end': 121, 'line': '    result = df.sum(axis=0)  # Sum in column direction (axis=0)\n'}]}, 'char_changes': {'deleted': [{'char_start': 15, 'char_end': 29, 'chars': 'arr = np.array'}, {'char_start': 67, 'char_end': 69, 'cha

problem
1. Comment, function name, and line space are also recognized as changes.
2.

In [16]:
# numpy-main-test-parsed-functions.json

In [65]:
import random
import json

In [66]:
json_file_path = 'numpy-main-test-parsed-functions.json'  

with open(json_file_path, 'r') as file:
    json_data = file.read()

numpy_parsed_data = json.loads(json_data)

print(len(numpy_parsed_data))

7138


In [67]:
json_file_path = 'pandas-main-test-parsed-functions.json'  

with open(json_file_path, 'r') as file:
    json_data = file.read()

pandas_parsed_data = json.loads(json_data)

print(len(pandas_parsed_data))

19520


In [36]:
import re

def reduce_multiple_newlines(input_string):
    return re.sub(r'\n+', '\n', input_string)

def remove_comments_and_whitespace(code):
    code_without_comments = re.sub(r'#.*$', '', code, flags=re.MULTILINE)
    return code_without_comments

In [68]:
all_values = list(numpy_parsed_data.values())

one_numpy = all_values[random.randint(0, len(numpy_parsed_data) - 1)]

one_numpy = reduce_multiple_newlines(one_numpy)

print(one_numpy)

    def test_keepdims_out(self, q, axis):
        d = np.ones((3, 5, 7, 11))
        # Randomly set some elements to NaN:
        w = np.random.random((4, 200)) * np.array(d.shape)[:, None]
        w = w.astype(np.intp)
        d[tuple(w)] = np.nan
        if axis is None:
            shape_out = (1,) * d.ndim
        else:
            axis_norm = normalize_axis_tuple(axis, d.ndim)
            shape_out = tuple(
                1 if i in axis_norm else d.shape[i] for i in range(d.ndim))
        shape_out = np.shape(q) + shape_out
        out = np.empty(shape_out)
        result = np.nanpercentile(d, q, axis=axis, keepdims=True, out=out)
        assert result is out
        assert_equal(result.shape, shape_out)


In [69]:
all_values = list(pandas_parsed_data.values())

one_pandas = all_values[random.randint(0, len(pandas_parsed_data) - 1)]

one_pandas = reduce_multiple_newlines(one_pandas)

print(one_pandas)

def by_row(request):
    return request.param


In [70]:
file_name = "temp.py"
src_before = one_pandas
src_after = one_numpy
diff = generate_diff(one_pandas, one_numpy)
result = parse_diff(file_name, src_before, src_after, diff)

In [71]:
print(diff)

@@ -1,2 +1,17 @@
-def by_row(request):
-    return request.param
+    def test_keepdims_out(self, q, axis):
+        d = np.ones((3, 5, 7, 11))
+        # Randomly set some elements to NaN:
+        w = np.random.random((4, 200)) * np.array(d.shape)[:, None]
+        w = w.astype(np.intp)
+        d[tuple(w)] = np.nan
+        if axis is None:
+            shape_out = (1,) * d.ndim
+        else:
+            axis_norm = normalize_axis_tuple(axis, d.ndim)
+            shape_out = tuple(
+                1 if i in axis_norm else d.shape[i] for i in range(d.ndim))
+        shape_out = np.shape(q) + shape_out
+        out = np.empty(shape_out)
+        result = np.nanpercentile(d, q, axis=axis, keepdims=True, out=out)
+        assert result is out
+        assert_equal(result.shape, shape_out)


In [72]:
print(result)

[{'func_name': 'by_row', 'func_src_before': 'def by_row(request):\n    return request.param', 'func_src_after': '    def test_keepdims_out(self, q, axis):\n        d = np.ones((3, 5, 7, 11))\n        # Randomly set some elements to NaN:\n        w = np.random.random((4, 200)) * np.array(d.shape)[:, None]\n        w = w.astype(np.intp)\n        d[tuple(w)] = np.nan\n        if axis is None:\n            shape_out = (1,) * d.ndim\n        else:\n            axis_norm = normalize_axis_tuple(axis, d.ndim)\n            shape_out = tuple(\n                1 if i in axis_norm else d.shape[i] for i in range(d.ndim))\n        shape_out = np.shape(q) + shape_out\n        out = np.empty(shape_out)\n        result = np.nanpercentile(d, q, axis=axis, keepdims=True, out=out)\n        assert result is out\n        assert_equal(result.shape, shape_out)', 'line_changes': {'deleted': [{'line_no': 1, 'char_start': 0, 'char_end': 21, 'line': 'def by_row(request):\n'}], 'added': [{'line_no': 1, 'char_start

In [46]:
numpy_code = """
import numpy as np

def calculate_row_means_numpy(matrix):
    # Calculate the mean of each row in the matrix
    row_means = np.mean(matrix, axis=1)
    
    # Extract rows where the mean is greater than 0.5
    selected_rows = matrix[row_means > 0.5]

    # Return the extracted rows and the mean of each row
    return selected_rows, row_means
"""

pandas_code = """
import pandas as pd

def calculate_row_means_pandas(df):
    # Calculate the mean of each row in the DataFrame
    row_means = df.mean(axis=1)

    # Extract rows where the mean is greater than 0.5
    mean_condition = df.mean(axis=1) > 0.5
    selected_rows = df[mean_condition]

    # Return the extracted rows and the mean of each row
    return selected_rows, row_means
"""

In [76]:
import ast
import astunparse

def parse_code(code):
    return ast.parse(code)

def extract_function_info(ast_tree):
    functions = []
    for node in ast.walk(ast_tree):
        if isinstance(node, ast.FunctionDef):
            function_info = {
                'name': node.name,
                'args': [arg.arg for arg in node.args.args],
                'body': astunparse.unparse(node).strip().split('\n'),
            }
            functions.append(function_info)
    return functions

numpy_ast = parse_code(numpy_code)
pandas_ast = parse_code(pandas_code)

numpy_functions = extract_function_info(numpy_ast)
pandas_functions = extract_function_info(pandas_ast)

In [77]:
numpy_functions

[{'name': 'calculate_row_means_numpy',
  'args': ['matrix'],
  'body': ['def calculate_row_means_numpy(matrix):',
   '    row_means = np.mean(matrix, axis=1)',
   '    selected_rows = matrix[(row_means > 0.5)]',
   '    return (selected_rows, row_means)']}]

In [78]:
pandas_functions

[{'name': 'calculate_row_means_pandas',
  'args': ['df'],
  'body': ['def calculate_row_means_pandas(df):',
   '    row_means = df.mean(axis=1)',
   '    mean_condition = (df.mean(axis=1) > 0.5)',
   '    selected_rows = df[mean_condition]',
   '    return (selected_rows, row_means)']}]

In [79]:
numpy_functions[0]['body']

['def calculate_row_means_numpy(matrix):',
 '    row_means = np.mean(matrix, axis=1)',
 '    selected_rows = matrix[(row_means > 0.5)]',
 '    return (selected_rows, row_means)']

In [ ]:
# import difflib

# numpy_code = """\
# import numpy as np 

# def sum():
#     arr = np.array([[1, 2, 3], [4, 5, 6]])
#     result = np.sum(arr, axis=0)  # Sum in column direction (axis=0)
#     print(result)
# """

# pandas_code = """\
# import pandas as pd 

# def sum():
#     df = pd.DataFrame([[1, 2, 3], [4, 5, 6]])
#     result = df.sum(axis=0)  # Sum in column direction (axis=0)
#     print(result)
# """

# differ = difflib.Differ()
# diff = differ.compare(numpy_code.splitlines(), pandas_code.splitlines())

# # 변경된 부분만 추출
# changed_lines = [line[2:] for line in diff if line.startswith('+ ') or line.startswith('- ')]
# for line in changed_lines:
#     print(line)

# print(changed_lines)